In [1]:
import os
os.chdir('../../src/models')

import pandas as pd
import datetime
import pickle as pkl

import plotly.express as px
import plotly.graph_objects as go

from policies import costs

set3 = px.colors.qualitative.Set3

color_map_set3 = {
    'Lockdown':          set3[3],
    'Hard Quarantine':    set3[11],
    'Light Quarantine':   set3[1],
    'Social Distancing':  'rgb(204, 245, 175)',
    'Unrestricted':        set3[6]     
}

In [2]:
def make_SIR_graph(data):
    d_counts = pd.DataFrame([pd.Series(d).value_counts() for d in data])
    d_counts.fillna(0, inplace=True)

    fig = go.Figure()
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(d_counts[0]))

    fig.add_trace(go.Scatter(x=x, y=d_counts[-1]/55e3, name='removed',line_color='green'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[0]/55e3, name='susceptible', line_color='blue'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[1]/55e3, name='exposed',line_color ='orange'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[2]/55e3, name='infected', line_color = 'red'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[3]/55e3, name='hospitalized', line_color = 'purple'))
    fig.update_layout(hovermode='x')
    fig.show()
    
    return d_counts

def make_beds_graph(data, actions, step_size, title, color_map):
    fig = go.Figure()
    
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(data)+2)

    fig.add_trace(go.Scatter(x=x, y=data[3]/55e3, name='hospitalized', line_color = 'royalblue',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=data[1]/55e3, name='exposed', line_color = 'firebrick',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=len(data)*[0.0025], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2)))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)],
                y1=1,
                fillcolor=a,
                opacity=0.5,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(coloraxis = {'colorscale':'deep'}, xaxis={'showgrid': False},
                      yaxis = {'showgrid': False},
                      showlegend=True, title = title, hovermode="x")
    fig.update_layout( 
        title=f'Monte Carlo Tree Search actions',
        font=dict(
            size=18,
        )
    )
    
    fig.write_image(f"{title}.png",scale=3)
    fig.show()

In [3]:
files = ['lower_pr_13_06_50_4_exposedCost_1.pkl',
        'lower_pr_13_06_50_4_exposedCost_2.pkl']
results = []

for file in files:
    with open(file, 'rb') as f:
        results.append(pkl.load(f))

for i,r in enumerate(results):
    df = make_SIR_graph(r[0])
    actions_colors = list(map(color_map_set3.get,  r[1]))
    make_beds_graph(df, actions_colors, 7, files[i], color_map_set3)
    print(pd.Series(map(costs.get,  r[1])).sum())

FileNotFoundError: [Errno 2] No such file or directory: 'lower_pr_13_06_50_4_exposedCost_1.pkl'

In [ ]:
pd.Series(map(costs.get,  len(results[0][1])*['Hard Quarantine'])).sum()